<a href="https://colab.research.google.com/github/AksamitKarolina/Surface_Approximation/blob/master/SurfaceApproximationApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
#IMPORT__________________________________________________________________________________
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from numpy.linalg import inv

#STEPS___________________________________________________________________________________
sns.set()

#1 LOAD DATA

path = "Model.txt"
#try open a file
try:
    file = open(path, "r")
except:
    print("Cannot open the file")

data = np.array([], dtype='float64')

#read and save data
for rows in file:
    rows = rows.split("	")
    nr = rows[0]
    X = float(rows[1])
    Y = float(rows[2])
    dz = float(rows[3])
    elem = np.array([nr, X, Y, dz], dtype='float64')
    data = np.concatenate((data, elem), axis=0)

#change the size of an array, get: nr, x, y, dz array
elem_num = int(len(data) / 4)
points = data.reshape(1, elem_num, 4)

#2 DISPLAY RAW DATA

Nr_points = points[0][:, [0]]
X_points = points[0][:, [1]]
Y_points = points[0][:, [2]]
dZ_points = points[0][:, [3]]

#make data flat
Nr_points1 = Nr_points.flatten()
X_points1 = X_points.flatten()
Y_points1 = Y_points.flatten()
dZ_points1 = dZ_points.flatten()
points = points[0]

#display the plot
fig_raw_dat = px.scatter_3d(x=X_points1, y=Y_points1, z=dZ_points1,color=Nr_points,title="Raw data from the model", 
                            labels ={'x': 'X [m]', 'y': 'Y [m]', 'z': 'dZ [mm]', 'color': 'Point number'}, width=1200, height=800, text=Nr_points )
fig_raw_dat.show()

In [76]:
#3 READ MODEL STRUCTURE
def check_xy(el, sign_xy, id, elem, col_a, val_exp, X_points, Y_points):
      if el == "x":
        if sign_xy == "^":
          cond = True
          for i, e in enumerate(list(elem)):
            if i > id+1 and cond == True:
              if e.isdigit():
                val_exp += e
              else:
                cond == False     
          val_exp = float(val_exp)
          col_a = np.multiply(col_a, X_points**val_exp)
        else:
          if sign_xy == "y":
            col_a = np.multiply(X_points, Y_points)
          else:
            col_a = np.multiply(col_a, X_points)

      elif el == "y":
          if sign_xy == "^":
            cond = True
            for i, e in enumerate(list(elem)):
              if i > id+1 and cond == True:
                if e.isdigit():
                  val_exp += e
                else:
                  cond == False
            val_exp = float(val_exp)
            col_a = np.multiply(col_a, Y_points**val_exp)
          else:
              if sign_xy == "x":
                col_a = np.multiply(Y_points, X_points)
              else:
                col_a = np.multiply(col_a, Y_points)
      else:
        col_a = np.array([], dtype='float64')
      return col_a

#change the way of displaying numbers
np.set_printoptions(precision=3, suppress=True)


#4 PARAMETERS

#4.1 read a given model (TESTED MODEL)
#model_shape = "a[0] + a[1]x + a[2]y"
model_shape2 = "a[0] + a[1]x + a[2]y + a[3]x^2 + a[4]y^2 + a[5]xy"
#model_shape3 = "a[0] + a[1]y + a[2]x^2 + a[3]y^2 + a[4]xy + a[5]y^3"

#4.2 split the model
model_split = model_shape2.split(sep=" ")

#4.3 check number of parameters a
num_a =0

A = np.array([], dtype='float64')

model_changed = model_shape2.replace("^","**")
model_changed = model_changed.replace("x","*X_points")
model_changed = model_changed.replace("y","*Y_points")
model_changed = model_changed.replace("a","B")

i=0
B = np.ones((elem_num,1), dtype='float64')
for index, element in enumerate(model_changed.split(" ")):
  if "B" in element:
    elem = eval(element)
    if len(elem) < elem_num:
      elem = B
    A = np.append(A, elem)
A = np.reshape(A, (elem_num, -1), 'F')

#4.4 L
L = points[:, [3]]


#5 LEAST SQUARE METHOD
#5.1 X
X = np.dot(inv(np.dot(np.transpose(A), A)), np.dot(np.transpose(A),L))

#5.2 AX -> predicted dz, V - deviations
AX = np.dot(A, X)
V = AX - L

#5.3 Accuracy, covariance..
sum_VV = np.sum(V**2)
m0 = np.sqrt(sum_VV/(np.shape(A)[0]-np.shape(A)[1]))
cov_X = m0**2*(inv(np.dot(np.transpose(A), A)))
sum_L_ = np.sum((L - np.mean(L))**2)
R2 = 1 - sum_VV/sum_L_
diag = np.diagonal(cov_X)
m_a = np.sqrt(diag)


#6 PREDICTION
x_pred = 15
y_pred = 15

model_pred = model_shape2.replace("^","**")
model_pred = model_pred.replace("x","*x_pred")
model_pred = model_pred.replace("y","*y_pred")
model_pred = model_pred.replace("a","X")
  
dz_pred = eval(model_pred)

#PREDICTION ACCURACY
model_pred = model_pred.replace("a","X")
F = np.array([], dtype="float64")
i=0
for index, element in enumerate(model_pred.split(" ")):
  if "X" in element:
    elem = eval(element)/X[i]
    F = np.append(F, elem)
    i+=1

m_dz_pred = np.sqrt(np.dot((np.dot(np.transpose(F), cov_X)), F))


#7 PLOTS
X_points =X_points.flatten()
Y_points = Y_points.flatten()
AX = AX

#Grid type the size
start_grid = 0
stop_grid = 100
num_grid = 101

#create grid
x_pred_arr, y_pred_arr = np.linspace(start_grid, stop_grid, num_grid), np.linspace(start_grid, stop_grid, num_grid)
# calculate dz values for the model

model_pred = model_shape2.replace("^","**")
model_pred = model_pred.replace("x","*x_pred")
model_pred = model_pred.replace("y","*y_pred")
model_pred = model_pred.replace("a","X")

dz_pred_arr = np.array([], dtype='float64') 
for xi in x_pred_arr:
  x_pred = xi
  for yi in y_pred_arr:
    y_pred = yi
    dz_pred = eval(model_pred)
    dz_pred_arr = np.append(dz_pred_arr, dz_pred)

dz_pred_arr = np.reshape(dz_pred_arr, (num_grid, num_grid))

fig = go.Figure(data=[go.Surface(z=dz_pred_arr, x=x_pred_arr, y=y_pred_arr)])
fig.update_layout(title='Surface Approximation - Model', autosize=False,
                  width=800, height=500, margin=dict(l=65, r=50, b=65, t=90))
fig.show()